In [31]:
#Loading all the needed Packages
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [76]:
#Load csv files 
HourlyMCP = pd.read_csv('../Data/ModelData/HourlyMCPDemo.csv', delimiter=',')
Fuel_Cost_Forecast_Normalized = pd.read_csv('../Data/ModelData/FuelCost_PriceDevelopment50years.csv', delimiter=',')
Generation_Asset_Data_New = pd.read_csv('../Data/ModelData/Generators_AssetData_New.csv', delimiter=',')
VRE_Generation_profile = pd.read_csv('../Data/ModelData/VRE_YearlyGenerationProfile_Normalized.csv', delimiter=',')

In [72]:
#show the first 5 rows of the data
#HourlyMCP.head()
#Fuel_Cost_Forecast_Normalized.head()
Generation_Asset_Data_New.head()

,Unit #,Zone,Technology,P_max (MW),P_min (MW),P_max current (MW),C_CapInv ($/MW),C_i ($/MWh),MaxInv (MW)
0,1,0,Coal,152,30.40,0,800000,30,200
1,2,0,Coal,152,30.40,0,800000,30,200
2,3,0,Coal,200,75.00,0,800000,30,200
3,4,0,Gas,200,206.85,0,820000,60,200
4,5,0,Gas,60,12.00,0,820000,60,200


In [73]:
#Define modelling time horizon
#Amount of years
Y = 1

#Amount of seasons in a year
S = 1

#Amount of hours in a season
H = 24

In [77]:
#Initialize the model
m = gp.Model("InvestmentModel")

#Define the decision variables
#Capacity Investment in invetment opportunity i defined by Generation_Asset_Data_New['Unit #']
Inv_NewCap = m.addVars(Generation_Asset_Data_New['Unit #'], vtype=GRB.CONTINUOUS, name="Inv_NewCap")
#Energy generated by investment opportunity i in hour h
P_NewCap = m.addVars(Generation_Asset_Data_New['Unit #'], H, vtype=GRB.CONTINUOUS, name="P_NewCap")

#Extract the values from the pandas Series
HourlyMCP_values = HourlyMCP['MCP'].values
C_CapInv_values = Generation_Asset_Data_New['C_CapInv ($/MW)'].values
VarCost_values = Generation_Asset_Data_New['C_i ($/MWh)'].values

#Define the objective function
#Objective function is to maximize the profit
m.setObjective(gp.quicksum((P_NewCap[i,h] * (HourlyMCP_values[h] - VarCost_values[j])) for j, i in enumerate(Generation_Asset_Data_New['Unit #']) for h in range(H)) - gp.quicksum(Inv_NewCap[i] * C_CapInv_values[j] for j, i in enumerate(Generation_Asset_Data_New['Unit #'])), GRB.MAXIMIZE)

#Define the constraints
#Investment constraint
m.addConstrs((Inv_NewCap[i] >= 0 for i in Generation_Asset_Data_New['Unit #']), "InvestmentMin")
m.addConstrs((Inv_NewCap[i] <= Generation_Asset_Data_New.loc[Generation_Asset_Data_New['Unit #'] == i, 'MaxInv (MW)'].values[0] for i in Generation_Asset_Data_New['Unit #']), "InvestmentMax")

#Budget constraint
m.addConstr(gp.quicksum(Inv_NewCap[i] * C_CapInv_values[j] for j, i in enumerate(Generation_Asset_Data_New['Unit #'])) <= 50000000, "Budget")

#Generation constraint
m.addConstrs((P_NewCap[i,h] <= Inv_NewCap[i] for i in Generation_Asset_Data_New['Unit #'] for h in range(H)), "GenerationMax")
m.addConstrs((P_NewCap[i,h] >= 0 for i in Generation_Asset_Data_New['Unit #'] for h in range(H)), "GenerationMin")

#Solve the model
m.optimize()

# Check if the model found an optimal solution
if m.status == GRB.OPTIMAL:
    # Print the optimal solution
    for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
    print('Obj:', m.objVal)
else:
    print("No optimal solution found. Status code:", m.status)

# Save the optimal solution to a df
# First df containts the investment decisions
df1 = pd.DataFrame(columns=['Unit #', 'Investment'])
df1['Unit #'] = Generation_Asset_Data_New['Unit #']
df1['Investment'] = [Inv_NewCap[i].x for i in Generation_Asset_Data_New['Unit #']]

# Second df contains the generation decisions
df2 = pd.DataFrame(columns=['Unit #', 'Hour', 'Generation'])
df2['Unit #'] = [i for i in Generation_Asset_Data_New['Unit #'] for h in range(H)]
df2['Hour'] = [h for i in Generation_Asset_Data_New['Unit #'] for h in range(H)]
df2['Generation'] = [P_NewCap[i,h].x for i in Generation_Asset_Data_New['Unit #'] for h in range(H)]

print(df1)
print(df2)





Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 801 rows, 400 columns and 1200 nonzeros
Model fingerprint: 0x3a21598b
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [2e+01, 2e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 5e+07]
Presolve removed 801 rows and 400 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -0.000000000e+00
Inv_NewCap[1] 0
Inv_NewCap[2] 0
Inv_NewCap[3] 0
Inv_NewCap[4] 0
Inv_NewCap[5] 0
Inv_NewCap[6] 0
Inv_NewCap[7] 0
Inv_NewCap[8] 0
Inv_NewCap[9] 0
Inv_NewCap[10] 0
Inv_NewCap

In [56]:
m.write("model.lp")